In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"

import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

import index as myidx
import crystal_dd
import transform as mytrans

jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platforms", 'cpu')


from ImageD11.parameters import AnalysisSchema
from ImageD11.columnfile import columnfile

%matplotlib widget

In [2]:
# set up detector

In [3]:
pars = AnalysisSchema.from_default().geometry_pars_obj
pars.set('tilt_x', 0.00123)
pars.set('tilt_y', -0.0345)
pars.set('tilt_z', 0.02)
pars.set('chi', 1)
pars.set('wedge', -3)
pars.set('t_x', 1)
pars.set('t_y', 2)
pars.set('t_z', 3)

In [4]:
pars.get('wavelength')

0.2845704

In [5]:
# make some g-vectors
# make some hkls first

In [6]:
import importlib
importlib.reload(crystal_dd)

<module 'crystal_dd' from '/home/esrf/james1997a/Code/Anri/anri/sandbox/crystal_dd.py'>

In [7]:
struc = crystal_dd.structure.from_cif('ICSD_CollCode43416.cif')

In [8]:
# computes all rings, but we don't know the intensities yet
struc.make_hkls(dsmax=1.0, wavelength=pars.get('wavelength'))

In [9]:
struc.rings_dict

{0:      h    k    l       tth        ds   intensity
 2 -1.0  1.0  0.0  6.382942  0.391277  261.613687
 3  1.0 -1.0  0.0  6.382942  0.391277  261.613687
 4  0.0 -1.0  0.0  6.382942  0.391277  262.183587
 5  1.0  0.0  0.0  6.382942  0.391277  261.613687
 6 -1.0  0.0  0.0  6.382942  0.391277  261.613687
 7  0.0  1.0  0.0  6.382942  0.391277  262.183587,
 1:      h    k    l       tth        ds   intensity
 8  0.0  0.0 -2.0  6.965661  0.426955  978.360958
 9  0.0  0.0  2.0  6.965661  0.426955  978.360958,
 2:       h    k    l       tth        ds   intensity
 10  1.0  0.0  1.0  7.272274  0.445724  707.931273
 11  0.0 -1.0  1.0  7.272274  0.445724  707.417717
 12  0.0 -1.0 -1.0  7.272274  0.445724  707.417717
 13  1.0  0.0 -1.0  7.272274  0.445724  707.931273
 14 -1.0  0.0 -1.0  7.272274  0.445724  707.931273
 15  0.0  1.0 -1.0  7.272274  0.445724  707.417717
 16  0.0  1.0  1.0  7.272274  0.445724  707.417717
 17 -1.0  0.0  1.0  7.272274  0.445724  707.931273
 18 -1.0  1.0  1.0  7.272274  

In [10]:
struc.ringds

array([0.39127669, 0.4269554 , 0.44572432, 0.57912724, 0.6777111 ,
       0.75050117, 0.78255337, 0.80098892, 0.81114888, 0.8539108 ,
       0.89144865, 0.93928755])

In [11]:
struc.ringhkls

{0.39127668517244407: array([[-1.,  1.,  0.],
        [ 1., -1.,  0.],
        [ 0., -1.,  0.],
        [ 1.,  0.,  0.],
        [-1.,  0.,  0.],
        [ 0.,  1.,  0.]]),
 0.42695540237344515: array([[ 0.,  0., -2.],
        [ 0.,  0.,  2.]]),
 0.44572432428969316: array([[ 1.,  0.,  1.],
        [ 0., -1.,  1.],
        [ 0., -1., -1.],
        [ 1.,  0., -1.],
        [-1.,  0., -1.],
        [ 0.,  1., -1.],
        [ 0.,  1.,  1.],
        [-1.,  0.,  1.],
        [-1.,  1.,  1.],
        [ 1., -1., -1.],
        [ 1., -1.,  1.],
        [-1.,  1., -1.]]),
 0.579127239883781: array([[-1.,  0., -2.],
        [ 1., -1.,  2.],
        [-1.,  1., -2.],
        [ 0.,  1., -2.],
        [ 0.,  1.,  2.],
        [ 0., -1., -2.],
        [ 1.,  0.,  2.],
        [ 0., -1.,  2.],
        [ 1., -1., -2.],
        [ 1.,  0., -2.],
        [-1.,  1.,  2.],
        [-1.,  0.,  2.]]),
 0.677711098535805: array([[ 1.,  1.,  0.],
        [-1., -1.,  0.],
        [ 2., -1.,  0.],
        [-1.,  2

In [12]:
struc.ringmult

array([ 6,  2, 12, 12,  6, 12,  6, 12, 12,  2, 12, 12])

In [13]:
B = struc.B

In [14]:
from scipy.spatial.transform import Rotation as R
U_in = R.random(1).as_matrix()[0]

In [15]:
U_in

array([[-0.33675275, -0.93946952, -0.06320293],
       [ 0.42468674, -0.09163481, -0.90069098],
       [ 0.84038013, -0.33015161,  0.42983852]])

In [16]:
all_hkls = np.concatenate(list(struc.ringhkls.values()))

In [17]:
gve = (U_in @ B @ all_hkls.T).T

In [18]:
gve.shape

(106, 3)

In [19]:
# all we need to test indexing

In [20]:
# does ImageD11 index it?

In [21]:
import ImageD11.indexing
import ImageD11.unitcell
import ImageD11.grain

In [22]:
struc.lattice_parameters

(2.95111, 2.95111, 4.68433, 90.0, 90.0, 120.0)

In [23]:
idx = ImageD11.indexing.indexer(unitcell=ImageD11.unitcell.unitcell(struc.lattice_parameters, int(struc._sym.spacegroup_number)),
                                gv=gve,
                                wavelength=pars.get('wavelength'))

info: gv: [[-0.25246271 -0.11413597 -0.27628428]
 [ 0.25246271  0.11413597  0.27628428]
 [ 0.38422621 -0.05203404 -0.05253687]
 [-0.1317635   0.16617002  0.32882115]
 [ 0.1317635  -0.16617002 -0.32882115]
 [-0.38422621  0.05203404  0.05253687]
 [ 0.02698483  0.38455488 -0.18352188]
 [-0.02698483 -0.38455488  0.18352188]
 [-0.14525591 -0.02610742  0.42058209]
 [ 0.37073379 -0.24431148  0.03922407]
 [ 0.39771863  0.14024339 -0.14429781]
 [-0.11827108  0.35844746  0.23706021]
 [ 0.14525591  0.02610742 -0.42058209]
 [-0.37073379  0.24431148 -0.03922407]
 [-0.39771863 -0.14024339  0.14429781]
 [ 0.11827108 -0.35844746 -0.23706021]
 [-0.26595513 -0.30641341 -0.18452334]
 [ 0.26595513  0.30641341  0.18452334]
 [ 0.23897029 -0.07814146  0.36804522]
 [-0.23897029  0.07814146 -0.36804522]
 [ 0.15874833  0.21838486 -0.51234303]
 [ 0.22547788 -0.2704189   0.45980616]
 [-0.22547788  0.2704189  -0.45980616]
 [-0.35724138  0.43658892 -0.13098501]
 [-0.41121104 -0.33252083  0.23605875]
 [ 0.41121104  

In [24]:
idx.assigntorings()

info: Assign to rings, maximum d-spacing considered: 0.939288
info: Ring assignment array shape (106,)
info: Ring     (  h,  k,  l) Mult  total indexed to_index  ubis  peaks_per_ubi   tth
info: Ring 11  ( -1,  0, -4)   12     12       0       12   N/A     N/A  15.36
info: Ring 10  ( -2,  0, -2)   12     12       0       12   N/A     N/A  14.57
info: Ring 9   (  0,  0, -4)    2      2       0        2   N/A     N/A  13.96
info: Ring 8   (  0, -2, -1)   12     12       0       12   N/A     N/A  13.26
info: Ring 7   ( -1, -1, -2)   12     12       0       12   N/A     N/A  13.09
info: Ring 6   (  0, -2,  0)    6      6       0        6   N/A     N/A  12.79
info: Ring 5   ( -1,  0, -3)   12     12       0       12   N/A     N/A  12.26
info: Ring 4   ( -1, -1,  0)    6      6       0        6   N/A     N/A  11.07
info: Ring 3   (  0, -1, -2)   12     12       0       12   N/A     N/A  9.45
info: Ring 2   ( -1,  0, -1)   12     12       0       12   N/A     N/A  7.27
info: Ring 1   (  0,  0,

In [25]:

idx.ring_1 = 2
idx.ring_2 = 7
# idx.score_all_pairs()
idx.find()
# idx.scorethem()
# grains = [ImageD11.grain.grain(ubi) for ubi in idx.ubis]
# U_id11 = grains[0].U
# print(U_id11)

info: hkls of rings being used for indexing
info: Ring 1: [(-1, 0, -1), (0, -1, -1), (-1, 1, -1), (1, -1, -1), (0, 1, -1), (1, 0, -1), (-1, 0, 1), (0, -1, 1), (-1, 1, 1), (1, -1, 1), (0, 1, 1), (1, 0, 1)]
info: Ring 2: [(-1, -1, -2), (-1, -1, 2), (1, -2, -2), (-2, 1, -2), (1, -2, 2), (-2, 1, 2), (2, -1, -2), (-1, 2, -2), (2, -1, 2), (-1, 2, 2), (1, 1, -2), (1, 1, 2)]
info: Possible angles and cosines between peaks in rings:
info: 26.035139 0.898525
info: 67.173920 0.387935
info: 75.208938 0.255295
info: 104.791062 -0.255295
info: 112.826080 -0.387935
info: 153.964861 -0.898525
info: Number of peaks in ring 1: 12
info: Number of peaks in ring 2: 12
info: Minimum number of peaks to identify a grain 10
info: Number of trial orientations generated 12
info: Time taken 0.000032 /s


In [26]:
ringds1 = struc.ringds[idx.ring_1]
ringds2 = struc.ringds[idx.ring_2]

hkls1 = struc.ringhkls[ringds1]
hkls2 = struc.ringhkls[ringds2]

In [27]:
# ok so ImageD11 works
# now we need to

In [28]:
# peaks -> gvecs -> assign to rings -> score all pairs

In [47]:
import importlib
importlib.reload(myidx)

<module 'index' from '/home/esrf/james1997a/Code/Anri/anri/sandbox/index.py'>

In [30]:
# myidx.filter_pairs(hkls1, hkls2, cangs_id11, B, BI)

In [31]:
ds = myidx.g_to_ds(gve)

In [32]:
%%time

# assign peaks to rings
ra = myidx.assign_peaks_to_rings(ds, struc.ringds, idx.ds_tol)

CPU times: user 47.7 ms, sys: 0 ns, total: 47.7 ms
Wall time: 38.4 ms


In [33]:
assert np.allclose(ra, idx.ra)

In [34]:
%%time

# (N1, N2) array of theoretical angles between hkls
cangs_id11 = ImageD11.unitcell.cosangles_many(hkls1, hkls2, idx.unitcell.gi)

CPU times: user 557 μs, sys: 0 ns, total: 557 μs
Wall time: 371 μs


In [35]:
%%time
cangs_me = myidx.anglehkls(hkls1, hkls2, struc.rmt)

CPU times: user 47.2 ms, sys: 89 μs, total: 47.3 ms
Wall time: 28.1 ms


In [36]:
assert np.allclose(cangs_id11, cangs_me)

In [37]:
# over npks, for each peak in ring 1, which peak in ring 2 does it like the best, if any < costol
res = myidx.find_gvector_cosangle_matches(gve, ra, idx.ring_1, idx.ring_2, hkls1, hkls2, struc.rmt, tol=1e-5)
resprint = np.column_stack((res['diffs'][res['mask']], res['i1'][res['mask']], res['i2'][res['mask']]))
print(resprint)
import pprint
pprint.pprint(idx.hits)
# print(np.unique(res['coses']))
# print(idx.cosangles)

[[ 0.  8. 56.]
 [ 0.  9. 57.]
 [ 0. 10. 57.]
 [ 0. 11. 56.]
 [ 0. 12. 56.]
 [ 0. 13. 57.]
 [ 0. 14. 57.]
 [ 0. 15. 56.]
 [ 0. 16. 56.]
 [ 0. 17. 56.]
 [ 0. 18. 56.]
 [ 0. 19. 56.]]
[[0.0, 8, 56],
 [5.551115123125783e-17, 9, 57],
 [1.1102230246251565e-16, 10, 58],
 [0.0, 11, 66],
 [0.0, 12, 60],
 [5.551115123125783e-17, 13, 56],
 [1.1102230246251565e-16, 14, 56],
 [0.0, 15, 62],
 [0.0, 16, 64],
 [0.0, 17, 63],
 [0.0, 18, 60],
 [0.0, 19, 56]]


In [38]:
B = struc.B
BI = np.linalg.inv(struc.B)

In [39]:
%%time

bt_me = myidx.BTmat(hkls1[0], hkls2[1] ,B, BI)

CPU times: user 126 ms, sys: 0 ns, total: 126 ms
Wall time: 101 ms


In [40]:
%time

bt_id11 = ImageD11.unitcell.BTmat(hkls1[0], hkls2[1], B, BI)

CPU times: user 6 μs, sys: 0 ns, total: 6 μs
Wall time: 11.2 μs


In [41]:
assert np.allclose(bt_me, bt_id11)

In [48]:
UBI = np.linalg.inv(U_in @ B)
score_id11 = ImageD11.cImageD11.score(UBI, gve, 0.01)
score_me = myidx.score(UBI, gve, 0.01)
assert score_id11 == score_me

In [49]:
gobs_in = np.random.random((3,3))
UBI_out_id11 = gobs_in.copy()
ImageD11.cImageD11.quickorient(UBI_out_id11, bt_me)
UBI_me = myidx.quickorient(gobs_in[0], gobs_in[1], bt_me)
assert np.allclose(UBI_me, UBI_out_id11)

In [50]:
%%time

hab_me, c2ab_me, matrs_me = myidx.filter_pairs(hkls1, hkls2, cangs_id11, B, BI)

CPU times: user 771 ms, sys: 153 ms, total: 923 ms
Wall time: 811 ms


In [51]:
%%time

hab, c2ab, matrs = ImageD11.unitcell.filter_pairs(hkls1, hkls2, cangs_id11, B, BI)

CPU times: user 6.6 ms, sys: 7.94 ms, total: 14.5 ms
Wall time: 5.78 ms


# unitcell.orient

We take two gvectors, one in ring 1 and one in ring 2.  
To define a UBI, you need two g-vectors and the corresponding BT matrix from a pair of HKLs.  
We know the theoretical hkls for each ring, but we don't know which hkls exactly the g-vectors are.  
The first test we can do is whether the angle between the g-vectors matches the angle between the HKLs.  
This is performed by `indexing.indexer.find` in ImageD11.
So for a given g-vector pair, from its cosangle we can find many possible hkl pairs that match.  
However, a lot of those hkl pairs define the same UBI (or a UBI that indexes the same peaks, e.g. the same modulo symmetry).  
So, we apply `unitcell.filter_pairs` to keep only the hkl pair indices that define unique UBIs.  
This means fewer potential UBIs for each g-vector pair.  

In [52]:
# two g-vectors to test with
# taken from ring 1 and ring 2
gi1 = 0
gi2 = 10

hab_me, c2ab_me, matrs_me = myidx.filter_pairs(hkls1, hkls2, cangs_id11, B, BI)

hab_me = hab_me[~np.isnan(c2ab_me)]
matrs_me = matrs_me[~np.isnan(c2ab_me)]
c2ab_me = c2ab_me[~np.isnan(c2ab_me)]

for i in range(len(hab_me)):
    h1, h2 = hab_me[i][:3], hab_me[i][3:]
    BT = matrs_me[i]
    g1 = gve[ra == idx.ring_1][gi1]
    g2 = gve[ra == idx.ring_2][gi2]
    UBI = myidx.quickorient(g1, g2, BT)
    # print(UBI)
    print(i, h1, h2, c2ab_me[i], myidx.score(UBI, gve, 1e-8))

print(' ')
print('#####')
print(' ')

hab, c2ab, matrs = ImageD11.unitcell.filter_pairs(hkls1, hkls2, np.array(cangs_me), B, BI, tol=1e-5)

for i in range(len(hab)):
    h1, h2 = hab[i]
    BT = matrs[i]
    g1 = gve[ra == idx.ring_1][gi1]
    g2 = gve[ra == idx.ring_2][gi2]
    UBI = myidx.quickorient(g1, g2, BT)
    # print(UBI)
    print(i, h1, h2, c2ab[i], myidx.score(UBI, gve, 1e-8))

assert len(hab_me) == len(hab)
assert np.allclose(c2ab_me, c2ab)

0 [-1.  1. -1.] [ 2. -1.  2.] -0.8985250249520884 4
1 [-1.  1. -1.] [ 1. -2.  2.] -0.8985250249520884 4
2 [-1.  1. -1.] [ 1. -2. -2.] -0.38793516826370994 4
3 [-1.  1. -1.] [ 2. -1. -2.] -0.3879351682637099 10
4 [-1.  1. -1.] [1. 1. 2.] -0.2552949283441894 4
5 [-1.  1. -1.] [-1. -1.  2.] -0.2552949283441891 4
6 [-1.  1. -1.] [ 1.  1. -2.] 0.2552949283441891 4
7 [-1.  1. -1.] [-1. -1. -2.] 0.2552949283441894 4
8 [-1.  1. -1.] [-2.  1.  2.] 0.3879351682637099 106
9 [-1.  1. -1.] [-1.  2.  2.] 0.38793516826370994 10
10 [-1.  1. -1.] [-2.  1. -2.] 0.8985250249520884 4
11 [-1.  1. -1.] [-1.  2. -2.] 0.8985250249520884 4
 
#####
 
0 [-1.  1. -1.] [ 2. -1.  2.] -0.8985250249520886 4
1 [ 1. -1.  1.] [-2.  1. -2.] -0.8985250249520886 4
2 [ 1. -1. -1.] [-2.  1. -2.] -0.3879351682637101 10
3 [-1.  1.  1.] [ 2. -1.  2.] -0.3879351682637101 4
4 [ 1. -1.  1.] [-1. -1. -2.] -0.25529492834418954 4
5 [-1.  1. -1.] [1. 1. 2.] -0.25529492834418954 4
6 [-1.  1.  1.] [1. 1. 2.] 0.2552949283441891 4
7 [-1. 

Once `unitcell.orient` gets the reduced hkl pairs, we perform the following lookup (for negative crange):
1. Calculate the cosangle between the observed g-vectors
2. Get the best matching reduced hkl pair in angle
3. Get the BT for that hkl pair
4. Orient the g-vector pair with the BT
5. Return UBI, UB

In [53]:
@jax.jit
def orient_negrange(g1, g2, hkls1, hkls2, cangs, B, BI)
    # get the reduced hkl indices that define unique peaks
    hkl_indices = myidx.filter_pairs(hkls1, hkls2, cangs, B, BI)
    costheta = jnp.dot(g1, g2) / jnp.sqrt((g1 * g1).sum() * (g2 * g2).sum())
    

SyntaxError: expected ':' (2609050316.py, line 2)

In [54]:
gi1 = 0
gi2 = 10

g1 = gve[ra == idx.ring_1][gi1]
g2 = gve[ra == idx.ring_2][gi2]

costheta = np.dot(g1, g2) / np.sqrt((g1 * g1).sum() * (g2 * g2).sum())
print(costheta)

i = np.searchsorted(c2ab, costheta, side='left')

0.3879351682637098


In [55]:
print(i > 0)  # did we find a position in the sorted angles list?
print(i == len(c2ab))  # did we hit the end of the list (no match)?
print(np.abs(costheta - c2ab[i - 1]))  # is the difference between g-vector angle and the previous angle better than the angle we're at?

# if we're not at the start, and we're either at the end or the previous angle was better
# then take the previous angle
# otherwise take this one
# this seems stupid, searchsorted just screwing up
# what about just getting the index of minimum distance?

if i > 0 and (i == len(c2ab) or(np.abs(costheta - c2ab[i - 1]) < np.abs(costheta - c2ab[i]))):
    best = [i - 1, ]
else:
    best = [i, ]

print(best)

True
False
1.6653345369377348e-16
[9]


In [ ]:
np.argmin(np.abs(c2ab - costheta))

In [ ]:
c2ab[8], costheta